In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

In [ ]:
from urllib.request import urlopen
import json    

# with urlopen('https://raw.githubusercontent.com/Subhash9325/GeoJson-Data-of-Indian-States/master/Indian_States') as response:
#     counties = json.load(response)

with urlopen('https://raw.githubusercontent.com/nikhilkumarsingh/choropleth-python-tutorial/master/states_india.geojson') as response:
    counties = json.load(response)

print(counties["features"][0].keys())
# Note that the loaded geoJSON must have an outermost key id. Note key id is not present here

In [ ]:
for i in range(len(counties['features'])):
    # Loop to add 'id' key
    counties['features'][i]['id']=counties['features'][i]['properties']['st_nm']
#     counties['features'][i]['id']=counties['features'][i]['properties']['NAME_1']
    
print(counties["features"][0].keys())

In [ ]:
names = []
for i in range(len(counties['features'])):
    names.append(counties['features'][i]['id'])

In [ ]:
df = pd.read_csv("data/NCRB_ADSI-2020_Table_1A.9.csv")
df = df[(df['Category']=='State') + (df['Category']=='Union Territories')]

Note : In geoJSON data, there is no separate data for Ladakh and Jammu Kashmir. However, in csv file there are two separate rows. Therefore merging the rows of Ladakh and Jammu Kashmir in dataframe. Also in geoJSON file there are two places 'Daman & Diu' and 'Dadara & Nagar Havelli', however, there is only one row named 'D & N Haveli and Daman & Diu' in dataframe. Therefore a copy of that row is made and used for both the places.

In [ ]:
id1=df.index[df['State/UT/City'] == 'Jammu & Kashmir'].tolist()[0]
id2=df.index[df['State/UT/City'] == 'Ladakh'].tolist()[0]
temp = df.loc[[id1,id2]]
temp = pd.concat([temp, (temp.sum(axis=0)).to_frame().transpose()])
temp = temp.drop(labels=[id1,id2],axis=0)
temp = temp.rename(index={0:id1})
temp.loc[id1,'Si. No.'] = str(id1)
temp.loc[id1,'Category'] = 'Union Territories'
temp.loc[id1,'State/UT/City'] = 'Jammu & Kashmir & Ladakh'
df = df.drop(labels=[id1,id2])
df = pd.concat([df,temp])

id1=df.index[df['State/UT/City'] == 'D & N Haveli and Daman & Diu'].tolist()[0]
temp = df.loc[id1].to_frame().transpose()
temp['State/UT/City']='D & N Haveli and Daman & Diu-2'
df = pd.concat([df,temp])

df = df.sort_values('State/UT/City', axis=0)
df.head()

In [ ]:
df_cols = list(df.columns)
for i in range(3,len(df_cols)):
    df[df_cols[i]]=df[df_cols[i]].astype('int64')   

In [ ]:
names.remove('NCT of Delhi')
names.append('Delhi')
names.sort()

In [ ]:
# Create a dictionary to map state name of dataframe to geo location
df2geo = {}
df_name = list(df['State/UT/City'])
for i in range(df.shape[0]):
    df2geo[df.iloc[i,2]]=names[i]
    
df2geo['Delhi (UT)']='NCT of Delhi'
df['name2geo']=df['State/UT/City'].map(df2geo)

In [ ]:
fig = px.choropleth_mapbox(df, geojson=counties, locations='name2geo', 
                           color='Grand Total - Died',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 20.5937, "lon": 78.9629},
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
# fig.write_html("../_html/map_totalDied.html")

In [3]:
df = pd.read_csv("data/Motor_Transport_Statistics_1995_Table_1.5.csv")
df.head()

,Year,Road Accidents (In thousands),Causalities in Road Accidents (In thousands)_Persons killed,Causalities in Road Accidents (In thousands)_Persons Injured,Vehicles Population (In Lakhs)
0,1970,114.1,14.5,70.1,NaN
1,1971,120.2,15.0,70.7,18.65
2,1972,122.3,16.1,76.4,20.45
3,1973,121.6,17.6,79.3,21.09
4,1974,114.3,17.3,76.7,23.27


In [13]:
# print(df.columns)
fig = px.line(df, x=df.columns[0], y=df.columns[1:])
# for i in range(1,len(df.columns)):
#     fig = px.line(df, x=df.columns[0], y=df.columns[i])
fig.show()

In [15]:
fig.write_image("lines_Motor_Transport_Statistics_1995.png")

In [ ]:
from IPython.display import IFrame

IFrame(src='../_html/map_totalDied.html', width=700, height=600)

In [1]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_states_and_union_territories_of_India_by_population"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [5]:
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])

In [6]:
df

,Rank,State or union territory,Population,National Share (%),Decadal growth(2001–2012),Rural population,Percent rural,Urban population,Percent urban,Area[14],Density[a],Sex ratio
0,1 (S1),Uttar Pradesh,199812341,NaN,20.2%,155317278,NaN,44495063,NaN,"240,928 km2 (93,023 sq mi)","828/km2 (2,140/sq mi)",912
1,2 (S2),Maharashtra,112374333,NaN,16%,61556074,NaN,50818259,NaN,"307,713 km2 (118,809 sq mi)",365/km2 (950/sq mi),929
2,3 (S3),Bihar,104099452,NaN,25.4%,92341436,NaN,11758016,NaN,"94,163 km2 (36,357 sq mi)","1,102/km2 (2,850/sq mi)",918
3,4 (S4),West Bengal,91276115,NaN,13.8%,62183113,NaN,29093002,NaN,"88,752 km2 (34,267 sq mi)","1,029/km2 (2,670/sq mi)",953
4,5 (S5),Madhya Pradesh,72626809,NaN,16.3%,52557404,NaN,20069405,NaN,"308,245 km2 (119,014 sq mi)",236/km2 (610/sq mi),931
5,6 (S6),Tamil Nadu,72147030,NaN,15.6%,37229590,NaN,34917440,NaN,"130,051 km2 (50,213 sq mi)","555/km2 (1,440/sq mi)",996
6,7 (S7),Rajasthan,68548437,NaN,21.3%,51500352,NaN,17048085,NaN,"342,239 km2 (132,139 sq mi)",201/km2 (520/sq mi),928
7,8 (S8),Karnataka,61095297,NaN,15.6%,30069335,NaN,31025962,NaN,"191,791 km2 (74,051 sq mi)",319/km2 (830/sq mi),979
8,9 (S9),Gujarat,60439692,NaN,19.3%,34694609,NaN,25745083,NaN,"196,024 km2 (75,685 sq mi)",308/km2 (800/sq mi),919
9,10 (S10),Andhra Pradesh,"49,577,103[b]",NaN,11.0%,34966693,NaN,14610410,NaN,"162,968 km2 (62,922 sq mi)",303/km2 (780/sq mi),993
